In [ ]:
!pip install https://github.com/kpu/kenlm/archive/master.zip
!pip install pyctcdecode==0.3.0
!pip install datasets==2.4.0
!pip install transformers==4.21.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.5/553.5 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.0.0-cp310-cp310-linux_x86_64.whl size=3255279 sha256=7f2b25582a70de8c946283678456c49dcbde21992fd1da248bf89921f61231ff
  Stored in directory: /tmp/pip-ephem-wheel-cache-r77j6pbu/wheels/a5/73/ee/670fbd0cee8f6f0b21d10987cb042291e662e26e1a07026462
Successfully built kenlm
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.4/417.4 kB 17.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━

In [ ]:
target_lang="pl"

In [ ]:
from datasets import load_dataset

dataset = load_dataset("europarl_bilingual", lang1="en", lang2=target_lang, split="train")
dataset

Generating train split:   0%|          | 0/631160 [00:00<?, ? examples/s]

Dataset europarl_bilingual downloaded and prepared to /root/.cache/huggingface/datasets/europarl_bilingual/en-pl-lang1=en,lang2=pl/8.0.0/2ab0200e7729616bfd4a4df6bfb29b31746ceb5a59f8c75c02ca35e1ebead950. Subsequent calls will reuse this data.


Dataset({
    features: ['translation'],
    num_rows: 631160
})

In [ ]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\'\«\»\–\—\’\„\…]'  # change to the ignored characters of your fine-tuned model

In [ ]:
import re

def extract_text(batch):
  text = batch["translation"][target_lang]
  batch["text"] = re.sub(chars_to_ignore_regex, "", text.lower())
  return batch

In [ ]:
dataset = dataset.map(extract_text, remove_columns=dataset.column_names)
dataset

  0%|          | 0/631160 [00:00<?, ?ex/s]

Dataset({
    features: ['text'],
    num_rows: 631160
})

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
dataset.push_to_hub(f"{target_lang}_corpora_parliament_processed", split="train")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:2168: FutureWarning: `identical_ok` has no effect and is deprecated. It will be removed in 0.11.0.
  FutureWarning,


In [ ]:
!sudo apt install build-essential cmake libboost-system-dev libboost-thread-dev libboost-program-options-dev libboost-test-dev libeigen3-dev zlib1g-dev libbz2-dev liblzma-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
libboost-program-options-dev is already the newest version (1.65.1.0ubuntu1).
libboost-system-dev is already the newest version (1.65.1.0ubuntu1).
libboost-thread-dev is already the newest version (1.65.1.0ubuntu1).
libboost-test-dev is already the newest version (1.65.1.0ubuntu1).
libeigen3-dev is already the newest version (3.3.4-4).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
libbz2-dev is already the newest version (1.0.6-8.1ubuntu0.2).
liblzma-dev is already the newest version (5.2.2-1.3ubuntu0.1).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [ ]:
!wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz

--2022-09-12 15:25:16--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 35.196.63.85
Connecting to kheafield.com (kheafield.com)|35.196.63.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491888 (480K) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 480.36K  --.-KB/s    in 0.03s   

2022-09-12 15:25:17 (17.3 MB/s) - written to stdout [491888/491888]



In [ ]:
!mkdir kenlm/build && cd kenlm/build && cmake .. && make -j2
!ls kenlm/build/bin

mkdir: cannot create directory ‘kenlm/build’: File exists
build_binary  fragment	       lmplz			     query
count_ngrams  interpolate      phrase_table_vocab	     streaming_example
filter	      kenlm_benchmark  probing_hash_table_benchmark


In [ ]:
from datasets import load_dataset

username = "Osolon"

dataset = load_dataset(f"{username}/{target_lang}_corpora_parliament_processed", split="train")

with open("text.txt", "w") as file:
  file.write(" ".join(dataset["text"]))

In [ ]:

!kenlm/build/bin/lmplz -o 5 <"text.txt" > "5gram.arpa"

=== 1/5 Counting and sorting n-grams ===
Reading /content/text.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
tcmalloc: large alloc 1917870080 bytes == 0x563e8dfd4000 @  0x7ff3ff68f1e7 0x563e8d4ea7e2 0x563e8d4854fe 0x563e8d4642eb 0x563e8d450066 0x7ff3fd828c87 0x563e8d451baa
tcmalloc: large alloc 8950054912 bytes == 0x563f004da000 @  0x7ff3ff68f1e7 0x563e8d4ea7e2 0x563e8d4d980a 0x563e8d4da248 0x563e8d464308 0x563e8d450066 0x7ff3fd828c87 0x563e8d451baa
****************************************************************************************************
Unigram tokens 12732162 types 176430
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:2117160 2:1062533824 3:1992251008 4:3187601408 5:4648585728
tcmalloc: large alloc 4648591360 bytes == 0x563e8dfd4000 @  0x7ff3ff68f1e7 0x563e8d4ea7e2 0x563e8d4d980a 0x563e8d4da248 0x563e8d4648d7 0x563e8d450066 0x7ff3fd828c87 0x563e8d451baa
tcmalloc: large alloc 1992253440 bytes == 

In [ ]:
!head -20 5gram.arpa

\data\
ngram 1=176429
ngram 2=3479940
ngram 3=8360371
ngram 4=11037078
ngram 5=12044293

\1-grams:
-6.5279236	<unk>	0
0	<s>	-0.10998965
-3.1737332	działania	-0.6326064
-4.1882515	podjęte	-0.51456666
-1.9289724	w	-1.2787143
-4.3358526	wyniku	-0.17023015
-3.7356699	rezolucji	-0.45794237
-3.3033776	parlamentu	-0.59647286
-4.53472	patrz	-1.0881011
-6.3867164	protokól	-0.10998965
-4.5844903	składanie	-0.4925539
-3.9324148	dokumentów	-0.38084775


In [ ]:
with open("5gram.arpa", "r") as read_file, open("5gram_correct.arpa", "w") as write_file:
  has_added_eos = False
  for line in read_file:
    if not has_added_eos and "ngram 1=" in line:
      count=line.strip().split("=")[-1]
      write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
    elif not has_added_eos and "<s>" in line:
      write_file.write(line)
      write_file.write(line.replace("<s>", "</s>"))
      has_added_eos = True
    else:
      write_file.write(line)

In [ ]:
!head -20 5gram_correct.arpa

\data\
ngram 1=176430
ngram 2=3479940
ngram 3=8360371
ngram 4=11037078
ngram 5=12044293

\1-grams:
-6.5279236	<unk>	0
0	<s>	-0.10998965
0	</s>	-0.10998965
-3.1737332	działania	-0.6326064
-4.1882515	podjęte	-0.51456666
-1.9289724	w	-1.2787143
-4.3358526	wyniku	-0.17023015
-3.7356699	rezolucji	-0.45794237
-3.3033776	parlamentu	-0.59647286
-4.53472	patrz	-1.0881011
-6.3867164	protokól	-0.10998965
-4.5844903	składanie	-0.4925539


In [ ]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("Osolon/wav2vec2-large-xls-r-300m-pl")

In [ ]:
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

In [ ]:
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(
    labels=list(sorted_vocab_dict.keys()),
    kenlm_model_path="5gram_correct.arpa",
)

In [ ]:
from transformers import Wav2Vec2ProcessorWithLM

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

In [ ]:
!sudo apt-get install git-lfs tree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 0s (268 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falli

In [ ]:
from huggingface_hub import Repository

repo = Repository(local_dir="wav2vec2-large-xls-r-300m-pl", clone_from="Osolon/wav2vec2-large-xls-r-300m-pl")

Cloning https://huggingface.co/Osolon/wav2vec2-large-xls-r-300m-pl-colab into local empty directory.


Download file pytorch_model.bin:   0%|          | 16.0k/1.18G [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 3.30k/3.30k [00:00<?, ?B/s]

Clean file training_args.bin:  30%|###       | 1.00k/3.30k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/1.18G [00:00<?, ?B/s]

In [ ]:
processor_with_lm.save_pretrained("wav2vec2-large-xls-r-300m-pl")

In [ ]:
!tree -h wav2vec2-large-xls-r-300m-pl/

wav2vec2-large-xls-r-300m-pl-colab/
├── [  30]  added_tokens.json
├── [ 303]  alphabet.json
├── [2.0K]  config.json
├── [4.0K]  language_model
│   ├── [1.7G]  5gram_correct.arpa
│   ├── [  78]  attrs.json
│   └── [1.9M]  unigrams.txt
├── [ 262]  preprocessor_config.json
├── [1.2G]  pytorch_model.bin
├── [1.8K]  README.md
├── [ 679]  special_tokens_map.json
├── [ 382]  tokenizer_config.json
├── [3.3K]  training_args.bin
└── [ 452]  vocab.json

1 directory, 13 files


In [ ]:
!kenlm/build/bin/build_binary wav2vec2-large-xls-r-300m-pl/language_model/5gram_correct.arpa wav2vec2-large-xls-r-300m-pl/language_model/5gram.bin

Reading wav2vec2-large-xls-r-300m-pl-colab/language_model/5gram_correct.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


In [ ]:
!rm wav2vec2-large-xls-r-300m-pl/language_model/5gram_correct.arpa && tree -h wav2vec2-large-xls-r-300m-pl/

wav2vec2-large-xls-r-300m-pl-colab/
├── [  30]  added_tokens.json
├── [ 303]  alphabet.json
├── [2.0K]  config.json
├── [4.0K]  language_model
│   ├── [737M]  5gram.bin
│   ├── [  78]  attrs.json
│   └── [1.9M]  unigrams.txt
├── [ 262]  preprocessor_config.json
├── [1.2G]  pytorch_model.bin
├── [1.8K]  README.md
├── [ 679]  special_tokens_map.json
├── [ 382]  tokenizer_config.json
├── [3.3K]  training_args.bin
└── [ 452]  vocab.json

1 directory, 13 files


In [ ]:
repo.push_to_hub(commit_message="Upload lm-boosted decoder")

Upload file language_model/5gram.bin:   0%|          | 3.34k/737M [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/Osolon/wav2vec2-large-xls-r-300m-pl-colab
   74a6b4b..2934571  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/Osolon/wav2vec2-large-xls-r-300m-pl-colab
   74a6b4b..2934571  main -> main



'https://huggingface.co/Osolon/wav2vec2-large-xls-r-300m-pl-colab/commit/293457132fff6419e9085bfa51403dfa09bc6a8a'